# **Procesado Digital de Imagen**

## Lab 3: Convolución y Filtros Lineales

2021 - Veronica Vilaplana - [GPI @ IDEAI](https://imatge.upc.edu/web/) Research group

-----------------

En esta práctica veremos cómo crear y utilizar filtros lineales y estudiaremos dos ejemplos de aplicación: eliminación de ruido y detección de contornos

-----------------

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

from skimage.io import imread, imshow
from skimage import transform as sktf
from skimage import data

#from skimage import io as skio

Definimos funciones para visualizar imágenes

In [ ]:
def display_image(img, title='', size=None):
  plt.gray()
  h = plt.imshow(img, interpolation='none')
  if size:
    dpi = h.figure.get_dpi()/size
    h.figure.set_figwidth(img.shape[1] / dpi)
    h.figure.set_figheight(img.shape[0] / dpi)
    h.figure.canvas.resize(img.shape[1] + 1, img.shape[0] + 1)
    h.axes.set_position([0, 0, 1, 1])
    h.axes.set_xlim(-1, img.shape[1])
    h.axes.set_ylim(img.shape[0], -1)
  plt.grid(False)
  plt.title(title)  
  plt.show()


Carga la imagen `lena.bmp` en Colab.
Leemos y visualizamos la imagen para verificar que se ha cargado correctamente.

In [ ]:
# Read the file from notebook disk
lena = imread('lena.bmp')
display_image(lena, size=1)

##1. Convolución. Respuesta al impulso
La función `convolve2d`de la librería `scipy.signal` realiza la convolución 2D de dos matrices.

En primer lugar vamos a definir el kernel de convolución como un array 2D numpy.
Creamos la respuesta impulsional de un filtro de promedio de tamaño 3x3 y filtramos la imagen.

In [ ]:
from scipy import signal

filt1 = np.ones([3, 3])
filt1 = filt1 / 9
# type of data
print(filt1.dtype)
print(filt1)

In [ ]:
lena_filt1 = signal.convolve2d(lena, filt1)
display_image(lena_filt1)

Observa los tipos de dato del kernel, la imagen original y del resultado de la convolución
Al calcular la convolución con un kernel con valores tipo float64, el resultado es tipo float64 también.

In [ ]:
print(lena.dtype)
print(filt1.dtype)
print(lena_filt1.dtype)

Observa el tamaño de la imagen original, el tamaño del kernel y el tamaño del resultado de la convolución.

Recuerda que la convolución de una matriz de tamaño longitud $M_1 \times N_1$ con otra secuencia de longitud $M_2 \times N_2$ es una matriz de tamaño $(M_1 + M_2 - 1) \times (N_1 + N_2 - 1)$


In [ ]:
print(lena.shape)
print(filt1.shape)
print(lena_filt1.shape)

Escribe los comandos necesarios para generar filtros de promedio de tamaño 7 (filt2) y de tamaño 11 (filt3). Filtra la imagen `lena` con estos filtros y comprueba el tamaño de la imagen filtrada. 

In [ ]:
# write your code here

<font color='blue'>Pregunta: 
Relaciona el tamaño de las imágenes de salida con el tamaño de la imagen de entrada
</font>

---
<font color='red'>Respuesta: 

</font>


<font color='blue'>Pregunta: 
Describe el efecto del filtrado para cada uno de los filtros
</font>

---
<font color='red'>Respuesta: 

</font>


<font color='blue'>Pregunta: 
Observa y explica el efecto de la convolución en los bordes de la imagen. 
</font>

---
<font color='red'>Respuesta: 

</font>


Si queremos que la imagen resultado de la convolución tenga el mismo tamaño que la imagen de entrada, podemos utilizar un parámetro de la función `convolve2d`

`convolve2d(original_image, impulse_response, mode='same')`

Aquí el parámetro `same` indica que la imagen de salida se recorta (descartando los bordes) para tener el mismo tamaño que la entrada.

Pero el tipo de dato sigue siendo float64, aunque los valores no estan en el rango $[0,1]$

In [ ]:
lena_filt1b = signal.convolve2d(lena, filt1,'same')
display_image(lena_filt1b)
print('output shape=',lena_filt1b.shape)
print('output type=',lena_filt1b.dtype)

print('Min value:',lena_filt1b.min())
print('Max value:',lena_filt1b.max())


Una alternativa para calcular la convolución de una imagen con un kernel, donde la salida es una imagen del mismo tamaño que la imagen de entrada, y tiene además el mismo tipo de dato, es utilizar la función `convolve` de la librería *Multidimensional Image Processing* de scipy (`scipy.ndimage`)

In [ ]:
from scipy import ndimage
lena_filt1c = ndimage.convolve(lena, filt1)
display_image(lena_filt1c)
print('output shape=',lena_filt1c.shape)
print('output type=',lena_filt1c.dtype)

##2. Filtros lineales

En el resto de la práctica trabajaremos con la librería Scikit-image, que tiene implementado un conjunto de filtros predefinidos. Puedes consultar los filtros disponibles aquí: https://scikit-image.org/docs/0.15.x/api/skimage.filters.html

En caso de querer filtrar la imagen con un kernel que no está implementado en la librería, siempre es posible definir el kernel como un nparray y utilizar la función `convolve` de `scipy.ndimage`, como en el ejemplo anterior.

Veamos algunos ejemplos de filtros.

####Filtro de promedio espacial

Es la alternativa de sckimage para los filtros de promedio espacial que aplicamos anteriormente con `convolve`.

`rank.mean(image, footprint)`

`footprint`: es la vecindad a considerar para calcular el promedio, expresado como ndarray con valores 0 o 1 (se calcula el promedio de los píxeles dentro de la vecindad). En este caso no hay que dar los valores del kernel, simplemente indicar con una matriz de 1 la extensión del kernel.

In [ ]:
from skimage import filters

footprint = np.ones([7,7])
lena_ave3 = filters.rank.mean(lena,footprint)
display_image(lena_ave3)
print(lena_ave3.shape)
print(lena_ave3.dtype)

#### Filtro Gaussiano
Un filtro paso-bajo implementado en scikit-image es el filtro Gaussiano. El kernel es una aproximación de una función Gaussiana. Los principales parámetros son `sigma`, la desviación estandard de la gaussiana, y `truncate`, que define con cuantas desviaciones estandard truncar el filtro (define en definitiva el tamaño del kernel).


In [ ]:
from skimage import filters

lena_gaus = filters.gaussian(lena,sigma=1, truncate=2)
display_image(lena_gaus)
print(lena_gaus.dtype)
print(lena_gaus.shape)

<font color='blue'>Pregunta: 
Prueba diferentes valores de `sigma`. Comenta el efecto del filtrado. 
</font>

---
<font color='red'>Respuesta: 

</font>

####Filtro Laplaciano

Uno de los filtros paso-alto de `scikit-image.filters` es el Laplaciano. El parámetro `ksize` define el tamaño del filtro ($ksize \times ksize$).


In [ ]:
from skimage import filters

lena_lap = filters.laplace(lena,ksize=3)
display_image(lena_lap)
print(lena_lap.dtype)
print(lena_lap.shape)

<font color='blue'>Pregunta: 
Comenta el efecto del filtrado. 
</font>

---
<font color='red'>Respuesta: 

</font>

####Filtros de gradiente

En la librería `filters` encontramos también otros filtros resaltadores de contornos que aproximan la primera derivada en dirección horizontal y vertical, como los filtros de Sobel y Prewit.
Por ejemplo, para Sobel encontramos los filtros `sobel_h` y `sobel_v`.

In [ ]:
lena_sobh = filters.sobel_h(lena)
display_image(lena_sobh)
lena_sobv = filters.sobel_v(lena)
display_image(lena_sobv)



<font color='blue'>Pregunta: 
Explica qué hace cada uno de estos dos filtros (`sobel_h` y `sobel_v`). Observa el tipo de dato y los valores máximo y mínimo del resultado (escribe los comandos necesarios)</font>

---
<font color='red'>Respuesta: 

</font>

In [ ]:
# your code here


##3. Eliminación de ruido

En este apartado utilizaremos filtros de promedio para eliminar el ruido de una iamgen

Cargamos a Colab y leemos la imagen `flor_ori.bmp`, le agregaremos ruido Gaussiano de media 0 y varianza 0.01

In [ ]:
from skimage import util, img_as_float

flor = imread('flor_ori.bmp')
display_image(flor, size=1)
print('original type:',flor.dtype)
print('original range:', flor.min(), flor.max())

#first, convert image data type from uint8 to float with values in [0,1]
florf = img_as_float(flor)
print('converted image type:',florf.dtype)
print('converted image range:', florf.min(), florf.max())

flor_noise = util.random_noise(florf, mode='gaussian', mean=0, var=0.01)
display_image(flor_noise, size=1)

print('noisy image type:',flor_noise.dtype)
print('noisy image range:', flor_noise.min(), flor_noise.max())

A continuación, escribe los comandos necesarios para filtrar la imagen ruidosa con filtros promediadores de diferentes tamaños (al menos tres filtros). Visualiza las imágenes filtradas

<font color='blue'>Pregunta: 
Compara la imagen original y las imágenes filtradas, y comenta el resultado del filtrado. Cuál de los filtros utilizados consideras que es el mejor?
</font>

---
<font color='red'>Respuesta: 

</font>

##4. Detección de contornos
Veamos ahora cómo utilizar filtros lineales para detectar contornos.

Carga en Colab y lee la imagen `circuit.bmp`

In [ ]:
circ = imread('circuit.bmp')
display_image(circ)

A continuación, escribe los comandos necesarios para aplicar a esta imagen los filtros de Sobel para realzar contornos horizontales y verticales.

Los pasos necesarios son:

1. Calcular gradientes horizontal y vertical
2. Combinar el resultado de ambos filtros según la siguiente expresión:

$ \nabla f = | g_x | + | g_y |$

Utiliza la función `abs` para calcular el valor absoluto de cada imagen filtrada.

3. Binarizar el resultado. 
Para esto puedes utilizar la siguiente expresión, donde `tresh` es el umbral elegido. 

`circ_bin = circ_sob > thres`

`display_image(circ_bin)`

Prueba diferentes valores hasta encontrar el que consideres más adecuado.


Nota: para obtener la imagen combinada también podrías utilizar directmente la función `filters.sobel`, que calcula la magnitud del gradiente, según la expresión <font color='red'>(pero no la utilizaremos en esta práctica, no la uses en este ejercicio) </font>


$ \nabla f = \sqrt {(g_x)^2  + (g_y)^2}$

In [ ]:
# Your code here...




<font color='blue'>Pregunta: 
Comenta a continuación la calidad de los resultados obtenidos y si has tenido problemas pra encontrar un umbral adecuado.
</font>

---
<font color='red'>Respuesta: 

</font>